## DMA Regression: Promo and Weather Effects by Hier2

#### Updated: 8/7/2019 to measure weather and promo effects

###### Modules
###### Import Libraries
###### Data Prep
###### Regression Model Loop
######     Volume Decomposition
######     Write Results to SQL

In [1]:
#import library statements
import pyodbc 
import pandas as pd
from pandas import DataFrame
import numpy as np
import statsmodels.api as sm
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
lm = LinearRegression(fit_intercept=False)

#user defined functions
def remove_ext_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-6*iqr
    fence_high = q3+6*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

### __Data Prep__

In [2]:
##Query 1 Select pos aggregates by week by DMA

server = 'adm-prod-sqldb.database.windows.net'
database = 'SMG-DW-PROD'
username = 'AzData'
password = 'AzureData1'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
sql = """
Select a.*
, dma+hier2 as model_key
From
(
Select
	dma
	,	dma_desc
    ,   region
	,	week_end_date
    ,   rpt_mth
    ,   hier_level2
    ,   case when hier_level2 = 'LAWN FERTILIZERS' then 1000 
            when hier_level2 = 'SOILS' then 2000 
            when hier_level2 = 'PLANT FOOD' then 3000 
            when hier_level2 = 'INDOOR INSECT' then 4000 
            when hier_level2 = 'GRASS SEED' then 5000
            when hier_level2 = 'OUTDOOR INSECT' then 6000 
            when hier_level2 = 'ROUNDUP' then 7000  
            when hier_level2 = 'WEED' then 8000 
            when hier_level2 = 'MULCH' then 9000 else null end as hier2
	,	sum(pos_sales_actual) as pos_dollars
	,	sum(pos_units_total) as pos_units
	,	sum(pos_sales_at_list) as pos_sales_at_list
	,	sum(discounted_dollars) as pos_discounted_dollars
	,	sum(discounted_units) as pos_discounted_units
	,	sum(stores_selling) as pos_stores
	,	max(CYear) as CYear
	,	max(CWeek) as CWeek
	,	avg(avg_max_temp) as avg_max_temp
	,	avg(avg_min_temp) as avg_min_temp
	,	avg(ttl_prcp) as avg_ttl_prcp
	,	sum(ttl_prcp) as ttl_prcp
	,    avg(avg_10yr_max_temp) as avg_10yr_max_temp
	,    avg(avg_10yr_min_temp) as avg_10yr_min_temp
	,    avg(avg_10yr_prcp) as avg_10yr_prcp
	,    avg(max_temp_diff) as max_temp_diff
	,    avg(min_temp_diff) as min_temp_diff
	,    avg(prcp_diff) as prcp_diff
	,    avg(max_temp_diff_pct) as max_temp_diff_pct
	,    avg(min_temp_diff_pct) as min_temp_diff_pct
	,    avg(prcp_diff_pct) as prcp_diff_pct
From rpt.PA__WTH_MODEL_CONSOLIDATED
Where CWeek <= 52 
and region = 'West'
--to run (,'West','South')
--partially complete ('Midwest')
--already run('Northeast' )
--and hier_level2 = 'OUTDOOR INSECT'
Group by dma,dma_desc,week_end_date,rpt_mth, hier_level2,region
) as a
"""
pos_df = pd.read_sql(sql,cnxn,parse_dates=['week_end_date'])
print(pos_df.head(3))

   dma        dma_desc region week_end_date     rpt_mth hier_level2  hier2  \
0  790  ALBUQ-SANTA FE   West    2007-10-06  2007-10-01  GRASS SEED   5000   
1  802          EUREKA   West    2007-10-06  2007-10-01  GRASS SEED   5000   
2  757           BOISE   West    2007-10-06  2007-10-01  GRASS SEED   5000   

   pos_dollars  pos_units  pos_sales_at_list  ...  avg_10yr_max_temp  \
0      3977.77        290            4345.39  ...          75.933441   
1       127.54          7             140.15  ...          62.752727   
2      3059.00        223            3605.01  ...          69.306666   

   avg_10yr_min_temp  avg_10yr_prcp  max_temp_diff  min_temp_diff  prcp_diff  \
0          48.568051       0.298408       1.915844       1.728377  -0.119837   
1          48.890909       0.798181      -3.362727      -5.220909  -0.038181   
2          46.909620       0.257120      -7.471666      -5.127953  -0.007120   

   max_temp_diff_pct  min_temp_diff_pct  prcp_diff_pct  model_key  
0        

In [4]:
len(pos_df.model_key.value_counts())

360

In [ ]:
#pos_df.tail()

In [4]:
##Variable Prep
## Add variables for POS Promo Cost and Avg. List Price
pos_df['pos_promo_cost']= pos_df.pos_sales_at_list - pos_df.pos_dollars
pos_df['avg_list_price'] = pos_df.pos_sales_at_list/pos_df.pos_units

## build early season and spring black friday date dummies
pos_df['early_season'] = np.where(pos_df.CWeek.isin(np.arange(9,17,1)),1,0)
pos_df['spr_blk_fri'] = np.where(pos_df.CWeek.isin(np.arange(12,15,1)),1,0)

##build early season weather variables
pos_df['es_max_temp_diff_pct']=pos_df.early_season*pos_df.max_temp_diff_pct
pos_df['roy_max_temp_diff_pct']=pos_df.max_temp_diff_pct - pos_df.es_max_temp_diff_pct

pos_df['es_prcp_diff_pct']=pos_df.early_season*pos_df.prcp_diff_pct
pos_df['roy_prcp_diff_pct']=pos_df.prcp_diff_pct - pos_df.es_prcp_diff_pct

##spring black friday by year
pos_df['spr_blk_fri_yr']=pos_df.spr_blk_fri*(pos_df['CYear'].loc[pos_df['CYear'] >= 2014])
pos_df['spr_blk_fri_yr']=pos_df['spr_blk_fri_yr'].fillna(0)
pos_df['spr_blk_fri_yr']=pos_df['spr_blk_fri_yr'].astype(int)

pos_df=pos_df.dropna()

In [ ]:
#test_loop = [1504,2504] #,'4504','5504','3504','6504','7504','8504','9504'
#pos_df2 = pos_df.loc[pos_df['dma']=='504',:]
#pos_df2.head()
#for model_key in pos_df['model_key'].unique():
#    tempdf = pos_df.loc[pos_df['model_key']==model_key,:]

In [ ]:
##pos_df2['model_key'].unique()

In [ ]:
#X1.head()

In [ ]:
#merged.head()

In [ ]:
#list(X1)

In [ ]:
#len(list(coeff_pivot))

In [5]:
for model_key in pos_df['model_key'].unique():
    print("Step 1: Define model key")
    mdl_df = pos_df.loc[pos_df['model_key']==model_key,:]
    print("n_obs: ",len(mdl_df))
    print("Model Key: ",mdl_df['model_key'].unique())
    print()
    print("Step 2: Define dep/indep vars")
    y1=mdl_df.pos_units/1000
    feat_cols = ['pos_promo_cost', 'CYear', 'CWeek','early_season','spr_blk_fri_yr','avg_max_temp','pos_stores','avg_list_price',
                 'es_max_temp_diff_pct','roy_max_temp_diff_pct','es_prcp_diff_pct','roy_prcp_diff_pct',]
    X1 = mdl_df[feat_cols]
    X2 = pd.get_dummies(X1, columns=['CWeek'])
    print("Number of Feature Columns: ", len(feat_cols))
    print("Step 2 Complete")
    print()
    
    print("Step 3: Build Model")
    model = sm.OLS(y1, X1)
    results = model.fit()
    summary = sm.OLS(y1, X1).fit().summary()
    dfout = mdl_df[['dma','dma_desc','region','week_end_date','rpt_mth','hier_level2','model_key','pos_units']]
    dfout['predicted'] = results.predict()*1000
    #print(summary)
    print("Step 3: Complete")
    print()
    
    print("Step 4: Build Model Decomp")
    ## build the model coeff specification file
    #results = model.fit()
    #model = sm.OLS(ty1, tx2).fit()
    model_spec = pd.DataFrame(columns=['model_key','features','coeffs','tvalues','pvalues'])
    model_spec['features']=results.params.index
    model_spec['coeffs']=results.params.values
    model_spec['pvalues']=results.pvalues.values
    model_spec['tvalues']=results.tvalues.values
    model_spec['sig_95_ci'] = np.where(model_spec['pvalues']<0.05, 1, 0)
    model_spec['model_key'] = mdl_df.model_key.max()
    model_spec.dtypes
    
    ##translate the model results into columner format
    mdl_coeff = model_spec[['model_key','features','coeffs']] 
    coeff_pivot = mdl_coeff.pivot(index='model_key',columns='features',values='coeffs')
    coeff_pivot = coeff_pivot.add_suffix('_coeff')
    coeff_pivot.reset_index(inplace=True)

    #coeff_pivot['model_key'] = dfout.model_key.max()
    merged = pd.merge(mdl_df, coeff_pivot, how='left', on=['model_key'])
    ### Decomp Step 4 Compute Impacts
    
    for i in list(X1):
        vol = i+'_vol_cont'
        coef = i+'_coeff'
        merged[vol] = merged[i] * merged[coef]

    ####Compute Month by Region Aggregates
    pred_mth_region = (merged.groupby(['region','rpt_mth','dma','hier_level2'])
                       .agg({'pos_units': 'sum',
                             'CYear_vol_cont': 'sum',
                             'CYear_vol_cont': 'sum',
                             'early_season_vol_cont': 'sum',
                             'spr_blk_fri_yr_vol_cont': 'sum',
                             'avg_max_temp_vol_cont': 'sum',
                             'pos_stores_vol_cont': 'sum',
                             'avg_list_price_vol_cont': 'sum',
                             'es_max_temp_diff_pct_vol_cont': 'sum',
                             'roy_max_temp_diff_pct_vol_cont': 'sum',
                             'es_prcp_diff_pct_vol_cont': 'sum',
                             'roy_prcp_diff_pct_vol_cont': 'sum',
                             })
                       .reset_index()
                       )
    
    ######Insert the DECOMP results into a SQL Table
    server = 'adm-prod-sqldb.database.windows.net'
    database = 'SMG-DW-PROD'
    username = 'AzData'
    password = 'AzureData1'
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)    
    cursor = cnxn.cursor()
    #cursor.execute('SELECT * FROM rpt.PA_MODEL_RESULTS_TEST')
    
    #currentdatetime = datetime.datetime.now()
    for index,row in pred_mth_region.iterrows():
        cursor.execute("INSERT INTO rpt.[PA__MACRO_MDL_NESTED_INPUTS_DECOMP]([region],[model_run_date],[rpt_mth],[dma],[hier_level2],[pos_units],[CYear_vol_cont],[early_season_vol_cont],[spr_blk_fri_yr_vol_cont],[avg_max_temp_vol_cont],[pos_stores_vol_cont],[avg_list_price_vol_cont],[es_max_temp_diff_pct_vol_cont],[roy_max_temp_diff_pct_vol_cont],[es_prcp_diff_pct_vol_cont],[roy_prcp_diff_pct_vol_cont]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row['region'],dt.date.today(),row['rpt_mth'],row['dma'],row['hier_level2'],row['pos_units'],row['CYear_vol_cont'],row['early_season_vol_cont'],row['spr_blk_fri_yr_vol_cont'],row['avg_max_temp_vol_cont'],row['pos_stores_vol_cont'],row['avg_list_price_vol_cont'],row['es_max_temp_diff_pct_vol_cont'],row['roy_max_temp_diff_pct_vol_cont'],row['es_prcp_diff_pct_vol_cont'],row['roy_prcp_diff_pct_vol_cont'])
        cnxn.commit()
    cursor.close()
    cnxn.close()
    
    ######Insert the MODEL COEFF results into a SQL Table
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)    
    cursor = cnxn.cursor()
    for index,row in model_spec.iterrows():
        cursor.execute("INSERT INTO [rpt].[PA__DMA_MODEL_SPECIFICATION_RESULTS]([model_key],[model_run_date],[features],[coeffs],[tvalues],[pvalues],[sig_95_ci]) values (?,?,?,?,?,?,?)",row['model_key'],dt.date.today(),row['features'],row['coeffs'],row['tvalues'],row['pvalues'],row['sig_95_ci'])
        cnxn.commit()
    cursor.close()
    cnxn.close()
    
    print("Model Complete: ", model_key)
 

Step 1: Define model key
n_obs:  604
Model Key:  [5790]

Step 2: Define dep/indep vars
Number of Feature Columns:  12
Step 2 Complete

Step 3: Build Model


C:\Users\U70470\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Step 3: Complete

Step 4: Build Model Decomp
Model Complete:  5790
Step 1: Define model key
n_obs:  584
Model Key:  [5802]

Step 2: Define dep/indep vars
Number of Feature Columns:  12
Step 2 Complete

Step 3: Build Model
Step 3: Complete

Step 4: Build Model Decomp
Model Complete:  5802
Step 1: Define model key
n_obs:  589
Model Key:  [5757]

Step 2: Define dep/indep vars
Number of Feature Columns:  12
Step 2 Complete

Step 3: Build Model
Step 3: Complete

Step 4: Build Model Decomp
Model Complete:  5757
Step 1: Define model key
n_obs:  562
Model Key:  [5771]

Step 2: Define dep/indep vars
Number of Feature Columns:  12
Step 2 Complete

Step 3: Build Model
Step 3: Complete

Step 4: Build Model Decomp
Model Complete:  5771
Step 1: Define model key
n_obs:  552
Model Key:  [5821]

Step 2: Define dep/indep vars
Number of Feature Columns:  12
Step 2 Complete

Step 3: Build Model
Step 3: Complete

Step 4: Build Model Decomp
Model Complete:  5821
Step 1: Define model key
n_obs:  595
Model Ke

In [ ]:
model_spec

In [ ]:
merged = pd.merge(mdl_df, coeff_pivot, how='left', on=['model_key'])
merged.head()

In [ ]:
#coeff_pivot.head()
merged = pd.merge(mdl_df, coeff_pivot, how='left', on=['model_key'])
merged.head()

for i in list(X1):
    vol = i+'_vol_cont'
    coef = i+'_coeff'
    merged[vol] = merged[i] * merged[coef]

merged.head()

In [ ]:

    #coeff_pivot.head()
    
    #coeff_pivot['model_key'] = dfout.model_key.max()
    merged = pd.merge(mdl_df, coeff_pivot, how='left', on=['model_key'])
    ### Decomp Step 4 Compute Impacts
    
    for i in list(X1):
        vol = i+'_vol_cont'
        coef = i+'_coeff'
        merged[vol] = merged[i] * merged[coef]

    ####Compute Month by Region Aggregates
    pred_mth_region = (merged.groupby(['region','rpt_mth','dma','hier_level2'])
                       .agg({'pos_units': 'sum',
                             'CYear_vol_cont': 'sum',
                             'CYear_vol_cont': 'sum',
                             'early_season_vol_cont': 'sum',
                             'spr_blk_fri_yr_vol_cont': 'sum',
                             'avg_max_temp_vol_cont': 'sum',
                             'pos_stores_vol_cont': 'sum',
                             'avg_list_price_vol_cont': 'sum',
                             'es_max_temp_diff_pct_vol_cont': 'sum',
                             'roy_max_temp_diff_pct_vol_cont': 'sum',
                             'es_prcp_diff_pct_vol_cont': 'sum',
                             'roy_prcp_diff_pct_vol_cont': 'sum',
                             })
                       .reset_index()
                       )
    
    ######Insert the results into a SQL Table
    server = 'adm-prod-sqldb.database.windows.net'
    database = 'SMG-DW-PROD'
    username = 'AzData'
    password = 'AzureData1'
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    
    cursor = cnxn.cursor()
    #cursor.execute('SELECT * FROM rpt.PA_MODEL_RESULTS_TEST')
    
    #currentdatetime = datetime.datetime.now()
    for index,row in pred_mth_region.iterrows():
        cursor.execute("INSERT INTO rpt.[PA__MACRO_MDL_NESTED_INPUTS_DECOMP]([region],[model_run_date],[rpt_mth],[dma],[hier_level2],[pos_units],[CYear_vol_cont],[early_season_vol_cont],[spr_blk_fri_yr_vol_cont],[avg_max_temp_vol_cont],[pos_stores_vol_cont],[avg_list_price_vol_cont],[es_max_temp_diff_pct_vol_cont],[roy_max_temp_diff_pct_vol_cont],[es_prcp_diff_pct_vol_cont],[roy_prcp_diff_pct_vol_cont]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row['region'],dt.date.today(),row['rpt_mth'],row['dma'],row['hier_level2'],row['pos_units'],row['CYear_vol_cont'],row['early_season_vol_cont'],row['spr_blk_fri_yr_vol_cont'],row['avg_max_temp_vol_cont'],row['pos_stores_vol_cont'],row['avg_list_price_vol_cont'],row['es_max_temp_diff_pct_vol_cont'],row['roy_max_temp_diff_pct_vol_cont'],row['es_prcp_diff_pct_vol_cont'],row['roy_prcp_diff_pct_vol_cont'])
        cnxn.commit()
    cursor.close()
    cnxn.close()
    
merged.head()
pred_mth_region.head()

In [ ]:
t1.model_key.value_counts()

In [ ]:
t1 = pos_df.loc[pos_df['model_key']==5504,:]
t1.head()

In [ ]:
## model key value counts
pos_df['model_key'].value_counts()

In [ ]:
#pos_df.loc[pos_df['model_key']==1535,:].head()

In [ ]:
y1=t1.pos_units/1000
feat_cols = ['pos_promo_cost', 'CYear', 'CWeek','early_season','spr_blk_fri_yr','avg_max_temp','pos_stores','avg_list_price',
             'es_max_temp_diff_pct','roy_max_temp_diff_pct','es_prcp_diff_pct','roy_prcp_diff_pct',]
X1 = t1[feat_cols]
X2 = pd.get_dummies(X1, columns=['CWeek'])


In [ ]:
model = sm.OLS(y1, X1)
results = model.fit()
summary = sm.OLS(y1, X1).fit().summary()
dfout = t1[['dma','dma_desc','region','week_end_date','rpt_mth','hier_level2','model_key','pos_units']]
dfout['predicted'] = results.predict()*1000

In [ ]:
t1.head()

In [ ]:
summary

In [ ]:
## build the model coeff specification file
#results = model.fit()
#model = sm.OLS(ty1, tx2).fit()
model_spec = pd.DataFrame(columns=['model_key','features','coeffs','tvalues','pvalues'])
#model_spec['model_key'] = testdf['model_key'].unique()
model_spec['features']=results.params.index
model_spec['coeffs']=results.params.values
model_spec['pvalues']=results.pvalues.values
model_spec['tvalues']=results.tvalues.values
model_spec['sig_95_ci'] = np.where(model_spec['pvalues']<0.05, 1, 0)
model_spec.dtypes

In [ ]:
##translate the model results into columner format
mdl_coeff = model_spec[['model_key','features','coeffs']] 
coeff_pivot = mdl_coeff.pivot(index='model_key',columns='features',values='coeffs')
coeff_pivot = coeff_pivot.add_suffix('_coeff')
coeff_pivot.reset_index(inplace=True)
#coeff_pivot.head()

In [ ]:
t1.model_key.value_counts()

In [ ]:
t1.head()

In [ ]:
coeff_pivot['model_key'] = t1.model_key.max()

In [ ]:
coeff_pivot.head()

In [ ]:
merged = pd.merge(t1, coeff_pivot, how='left', on=['model_key'])

In [ ]:
merged.head()

In [ ]:
### Decomp Step 4 Compute Impacts

In [ ]:
X1.columns
coeff_pivot.columns

In [ ]:
merged.columns

In [ ]:
len(coeff_pivot.columns)

In [ ]:
for i in list(X1):
    vol = i+'_vol_cont'
    coef = i+'_coeff'
    merged[vol] = merged[i] * merged[coef]
merged.head()

In [ ]:
##Compute Month by Region Aggregates
pred_mth_region = (merged.groupby(['region','rpt_mth','dma','hier_level2'])
              .agg({'pos_units': 'sum',
                    'CYear_vol_cont': 'sum',
                    'CYear_vol_cont': 'sum',
                    'early_season_vol_cont': 'sum',
                    'spr_blk_fri_yr_vol_cont': 'sum',
                    'avg_max_temp_vol_cont': 'sum',
                    'pos_stores_vol_cont': 'sum',
                    'avg_list_price_vol_cont': 'sum',
                    'es_max_temp_diff_pct_vol_cont': 'sum',
                    'roy_max_temp_diff_pct_vol_cont': 'sum',
                    'es_prcp_diff_pct_vol_cont': 'sum',
                    'roy_prcp_diff_pct_vol_cont': 'sum',
                    })
              .reset_index()
             )

In [ ]:
pred_mth_region.head()

In [ ]:
pred_mth_region.columns

In [ ]:
###Insert the results into a SQL Table
server = 'adm-prod-sqldb.database.windows.net'
database = 'SMG-DW-PROD'
username = 'AzData'
password = 'AzureData1'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

cursor = cnxn.cursor()
#cursor.execute('SELECT * FROM rpt.PA_MODEL_RESULTS_TEST')

#currentdatetime = datetime.datetime.now()
for index,row in pred_mth_region.iterrows():
    cursor.execute("INSERT INTO rpt.[PA__MACRO_MDL_NESTED_INPUTS_DECOMP]([region],[model_run_date],[rpt_mth],[dma],[hier_level2],[pos_units],[CYear_vol_cont],[early_season_vol_cont],[spr_blk_fri_yr_vol_cont],[avg_max_temp_vol_cont],[pos_stores_vol_cont],[avg_list_price_vol_cont],[es_max_temp_diff_pct_vol_cont],[roy_max_temp_diff_pct_vol_cont],[es_prcp_diff_pct_vol_cont],[roy_prcp_diff_pct_vol_cont]) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row['region'],dt.date.today(),row['rpt_mth'],row['dma'],row['hier_level2'],row['pos_units'],row['CYear_vol_cont'],row['early_season_vol_cont'],row['spr_blk_fri_yr_vol_cont'],row['avg_max_temp_vol_cont'],row['pos_stores_vol_cont'],row['avg_list_price_vol_cont'],row['es_max_temp_diff_pct_vol_cont'],row['roy_max_temp_diff_pct_vol_cont'],row['es_prcp_diff_pct_vol_cont'],row['roy_prcp_diff_pct_vol_cont'])
    cnxn.commit()
cursor.close()
cnxn.close()

In [ ]:
## build the model coeff specification file
#results = model.fit()
#model = sm.OLS(ty1, tx2).fit()
model_spec = pd.DataFrame(columns=['model_key','features','coeffs','tvalues','pvalues'])
#model_spec['model_key'] = testdf['model_key'].unique()
model_spec['features']=model.params.index
model_spec['coeffs']=model.params.values
model_spec['pvalues']=model.pvalues.values
model_spec['tvalues']=model.tvalues.values
model_spec['sig_95_ci'] = np.where(model_spec['pvalues']<0.05, 1, 0)
model_spec.dtypes

In [ ]:
results.params.index

In [ ]:
dfout.head()

In [ ]:
dfout.tail()

In [ ]:
### Run Model Loop
a1 = []
a2 = []
a3 = []
a4 = []
for model_key in pos_df['model_key'].unique():
    tempdf = pos_df.loc[pos_df['model_key']==model_key,:]
    train = tempdf.loc[tempdf['CYear'] <= 2019,:]
    #test = tempdf.loc[tempdf['CYear'] > 2017,:]
        
    #Define dependent variables
    y1 = train.pos_units/1000
    y2 = train.pos_dollars/1000
    
    #Define independent Variables
    feat_cols = ['pos_promo_cost', 'CYear', 'CWeek','early_season','spr_blk_fri_yr','avg_max_temp','pos_stores','avg_list_price',
             'es_max_temp_diff_pct','roy_max_temp_diff_pct','es_prcp_diff_pct','roy_prcp_diff_pct',] #'pct_dol_disct', 
    
    X1 = train[feat_cols]
    X2 = pd.get_dummies(X1, columns=['CWeek'])
    
    
    #### train the model
    model = sm.OLS(y1, X2)
    results = model.fit()
    summary = sm.OLS(y1, X2).fit().summary()
    dfout = train[['dma','dma_desc','region','week_end_date','rpt_mth','hier_level2','model_key','pos_units']]
    dfout['predicted'] = results.predict()*1000
    #dfout['predicted']=sm.OLS(y1.astype(float),X2.astype(float)).fit().predict()*1000
    
    
    a3.append(dfout)
    
    ### test holdout data
    #y1h = test.pos_units/1000
    
    #X1h = test[feat_cols]
    #X2h = pd.get_dummies(X1h, columns=['CWeek'])
        
    #holdout = test[['dma','dma_desc','region','week_end_date','rpt_mth','hier_level2','model_key','pos_units']]
    #holdout['predicted']=results.predict(X2h)*1000
    
    #a4.append(holdout)
    
pred_df = pd.concat(a3)
#holdout_df = pd.concat(a4)

In [ ]:
#len(pred_df['model_key'].value_counts())
pred_df.head()

In [ ]:
## Compute Error Terms
pred_df['error'] = pred_df.pos_units - pred_df.predicted
pred_df['abs_pct_error']=abs(pred_df.error/pred_df.predicted)
pred_df.groupby('model_key')['abs_pct_error'].mean()

In [ ]:
##Compute Month by Region Aggregates
pred_mth_region = (pred_df.groupby(['region','rpt_mth','hier_level2'])
              .agg({'pos_units': 'sum',
                   'predicted': 'sum',
                    })
              .reset_index()
              .rename(columns={'pos_units': 'pos_units',
                              'predicted': 'predicted_units'
                               })
             )

In [ ]:
pred_mth_region.head()

In [ ]:
#pred_mth_region.region.value_counts()
#pred_mth_region.hier_level2.value_counts()

In [ ]:
###Insert the results into a SQL Table
server = 'adm-prod-sqldb.database.windows.net'
database = 'SMG-DW-PROD'
username = 'AzData'
password = 'AzureData1'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

cursor = cnxn.cursor()
#cursor.execute('SELECT * FROM rpt.PA_MODEL_RESULTS_TEST')

#currentdatetime = datetime.datetime.now()
for index,row in pred_mth_region.iterrows():
    cursor.execute("INSERT INTO rpt.[PA__MACRO_MDL_NESTED_INPUTS]([region],[model_run_date],[rpt_mth],[hier_level2],[pos_units],[predicted_units]) values (?,?,?,?,?,?)",row['region'],dt.date.today(),row['rpt_mth'], row['hier_level2'],row['pos_units'],row['predicted_units'])
    cnxn.commit()
cursor.close()
cnxn.close()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.lineplot(x='rpt_mth', y='pos_units', color='g', data=pred_mth_region)
sns.lineplot(x='rpt_mth', y='predicted_units', color='b', data=pred_mth_region)

In [ ]:
pred_mth_region['pred_error']=pred_mth_region.pos_units - pred_mth_region.predicted_units

In [ ]:
pred_mth_region['abs_pct_error']=abs(pred_mth_region.pred_error/pred_mth_region.pos_units)

In [ ]:
pred_mth_region.head()

In [ ]:
pred_df['abs_pct_error']=abs(pred_df.error/pred_df.predicted)
pred_df.groupby('model_key')['abs_pct_error'].mean()

In [ ]:
pred_mth_region.to_csv('MIDWEST_GRASS_SEED.csv')

In [ ]:
pred_mth_dma = (pred_df.groupby(['model_key','rpt_mth'])
              .agg({'pos_units': 'sum',
                   'predicted': 'sum',
                    'dma': 'max',
                    'dma_desc': 'max',
                    'hier_level2': 'max'
                   })
              .reset_index()
              .rename(columns={'pos_units': 'pos_units',
                              'predicted': 'predicted_units',
                              'dma': 'dma',
                              'dma_desc': 'dma_desc',
                              'hier_level2': 'hier_level2'})
             )

In [ ]:
pred_mth_dma.head()

In [ ]:
#sql_test_df = pred_mth_dma
pred_mth_dma.hier_level2

In [ ]:
sql_test_df = pred_mth_dma[pred_mth_dma['hier_level2']=='SOILS']

In [ ]:
for model_key in sql_test_df['model_key'].unique():
    ###Insert the results into a SQL Table
    server = 'adm-prod-sqldb.database.windows.net'
    database = 'SMG-DW-PROD'
    username = 'AzData'
    password = 'AzureData1'
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

    cursor = cnxn.cursor()
    #cursor.execute('SELECT * FROM rpt.PA_MODEL_RESULTS_TEST')

    #currentdatetime = datetime.datetime.now()
    for index,row in sql_test_df.iterrows():
        cursor.execute("INSERT INTO rpt.PA__NESTED_MODEL_RESULTS([model_key],[model_run_date],[dma],[dma_desc],[rpt_mth],[hier_level2],[pos_units],[predicted_units]) values (?,?,?, ?,?,?,?,?)",row['model_key'],dt.date.today(),row['dma'] , row['dma_desc'], row['rpt_mth'], row['hier_level2'],row['pos_units'],row['predicted_units'])
        cnxn.commit()
    cursor.close()
    cnxn.close()
    print('SQL Load Complete:', model_key)

In [ ]:
pred_mth_dma.model_key.values()

In [ ]:
###Insert the results into a SQL Table
server = 'adm-prod-sqldb.database.windows.net'
database = 'SMG-DW-PROD'
username = 'AzData'
password = 'AzureData1'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

cursor = cnxn.cursor()
#cursor.execute('SELECT * FROM rpt.PA_MODEL_RESULTS_TEST')

#currentdatetime = datetime.datetime.now()
for index,row in pred_mth_dma.iterrows():
    cursor.execute("INSERT INTO rpt.PA__NESTED_MODEL_RESULTS([model_key],[model_run_date],[dma],[dma_desc],[rpt_mth],[hier_level2],[pos_units],[predicted_units]) values (?,?,?, ?,?,?,?,?)",row['model_key'],dt.date.today(),row['dma'] , row['dma_desc'], row['rpt_mth'], row['hier_level2'],row['pos_units'],row['predicted_units'])
    cnxn.commit()
cursor.close()
cnxn.close()